In [1]:
import xarray as xr
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_dataset(f"{dir_r}full_disc_region_2022.nc",
   ).sortby("points").chunk({"points": 2000})
ds

<xarray.Dataset> Size: 145GB
Dimensions:                                  (points: 86193, time: 8760)
Coordinates:
  * points                                   (points) int64 690kB 13653 ... 2...
  * time                                     (time) datetime64[ns] 70kB 2022-...
Data variables: (12/40)
    south_no_track_P_mp                      (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    west_no_track_P_mp                       (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    east_no_track_P_mp                       (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    1_track_P_mp                             (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    2_track_P_mp                             (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    bifacial_vertical_west_P_mp              (points, time) float32 3GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    ...                                       ...
    east_no_track_P_mp_total                 (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    1_track_P_mp_total                       (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    2_track_P_mp_total                       (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    bifacial_vertical_west_P_mp_total        (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    bifacial_vertical_east_P_mp_total        (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>
    bifacial_south_P_mp_total                (points, time) float64 6GB dask.array<chunksize=(2000, 8760), meta=np.ndarray>

In [78]:
# Cargamos datos.
dir_d = "/datos/rodr/temp/recurso_solar/duck_curve/"
points = pd.read_csv(f"{dir_d}conus_points.csv", index_col = 0)
points = points[points["REGION"].notnull()]
points["REGION"] = points["REGION"].astype(np.int16)
ds = points[["latitude", "longitude", "elevation", "timezone", "REGION",
    "potential_solar_park_zones", "built_surface"]
    ].to_xarray().rename({"index": "points"}).sel(points = 13653)
ds["time"] = pd.date_range("2022-01-01", "2023-01-01", freq = "1h")[:-1]
ds["Pressure"] = (1013.25 * (1 - 0.0065/288.15*ds["elevation"]) ** 5.25588
    ).astype(np.float32)
ds = ds.drop_vars("elevation")
# Convertimos a fecha.
ds["hour"] = (ds["time"].dt.hour).astype(
    np.float32)
ds["dayofyear"] = ds["time"].dt.dayofyear.astype(
    np.int16)
TZ = ds["timezone"]
lat = ds["latitude"]
lon = ds["longitude"]
# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day_Angle"] = (6.283185 * ( ds["dayofyear"] - 1 ) / 365
    ).astype(np.float32)
# Declinación.
ds["Declination"] = ((0.006918 - 0.399912 * np.cos(ds["Day_Angle"])
    + 0.070257*np.sin(ds["Day_Angle"])
    - 0.006758*np.cos(2*ds["Day_Angle"])
    + 0.000907*np.sin(2*ds["Day_Angle"])
    - 0.002697*np.cos(3*ds["Day_Angle"])
    + 0.00148*np.sin(3*ds["Day_Angle"])) * 180/np.pi
    ).astype(np.float32)
# Ecuación del tiempo.
ds["Time_Equation"] = ((0.000075 + 0.001868*np.cos(ds["Day_Angle"])
    - 0.032077*np.sin(ds["Day_Angle"])
    - 0.014615*np.cos(2*ds["Day_Angle"])
    -0.040849*np.sin(2*ds["Day_Angle"])) * 229.18
    ).astype(np.float32)
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * (ds["hour"]
    + ds["Time_Equation"]/60
    ).astype(np.float32)
# Posiciones del analema solar.
ds["Sz"] = (sin(lat)*sin(ds["Declination"])
    - cos(lat)*cos(ds["Declination"])
    *cos(ds["lon_subs"]-lon)
    ).astype(np.float32)
ds = ds.drop_vars(["lon_subs"])
# Factor b de Pérez
ds["b"] = ds["Sz"].clip(max = cos(85))
# Ángulo del cénit solar.
ds["Zenith_Angle"] = acos(ds["Sz"]).astype(np.float32)

ds.to_dataframe()["Zenith_Angle"].iloc[0:30]

time
2022-01-01 00:00:00     81.519676
2022-01-01 01:00:00     90.823875
2022-01-01 02:00:00    101.103951
2022-01-01 03:00:00    112.025429
2022-01-01 04:00:00    123.314301
2022-01-01 05:00:00    134.697021
2022-01-01 06:00:00    145.780914
2022-01-01 07:00:00    155.702286
2022-01-01 08:00:00    162.022232
2022-01-01 09:00:00    160.639923
2022-01-01 10:00:00    152.720184
2022-01-01 11:00:00    142.267532
2022-01-01 12:00:00    131.027405
2022-01-01 13:00:00    119.641479
2022-01-01 14:00:00    108.442963
2022-01-01 15:00:00     97.696877
2022-01-01 16:00:00     87.691429
2022-01-01 17:00:00     78.789886
2022-01-01 18:00:00     71.461411
2022-01-01 19:00:00     66.267349
2022-01-01 20:00:00     63.753998
2022-01-01 21:00:00     64.246277
2022-01-01 22:00:00     67.676933
2022-01-01 23:00:00     73.624771
2022-01-02 00:00:00     81.393372
2022-01-02 01:00:00     90.694962
2022-01-02 02:00:00    100.974190
2022-01-02 03:00:00    111.895630
2022-01-02 04:00:00    123.184669
2022-01-0

In [79]:
df["ghi"].iloc[0:30]

time
2022-01-01 00:00:00     12
2022-01-01 01:00:00      0
2022-01-01 02:00:00      0
2022-01-01 03:00:00      0
2022-01-01 04:00:00      0
2022-01-01 05:00:00      0
2022-01-01 06:00:00      0
2022-01-01 07:00:00      0
2022-01-01 08:00:00      0
2022-01-01 09:00:00      0
2022-01-01 10:00:00      0
2022-01-01 11:00:00      0
2022-01-01 12:00:00      0
2022-01-01 13:00:00      0
2022-01-01 14:00:00      0
2022-01-01 15:00:00      0
2022-01-01 16:00:00     16
2022-01-01 17:00:00    117
2022-01-01 18:00:00    223
2022-01-01 19:00:00    414
2022-01-01 20:00:00    463
2022-01-01 21:00:00    457
2022-01-01 22:00:00    392
2022-01-01 23:00:00    277
2022-01-02 00:00:00    130
2022-01-02 01:00:00      0
2022-01-02 02:00:00      0
2022-01-02 03:00:00      0
2022-01-02 04:00:00      0
2022-01-02 05:00:00      0
Name: ghi, dtype: int16

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar

dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
points = pd.read_csv(f"{dir_r}conus_points.csv").rename( columns = 
    {"Unnamed: 0": "points"}).set_index("points")[["latitude", "longitude",
    "timezone", "REGION", "potential_solar_park_zones", "built_surface"]
    ].to_xarray()
ds_0 = xr.open_dataset(f"{dir_r}full_disc_PV_2022.nc").chunk({"time": 3000})
ds = xr.merge([ds_0, points])

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]
# Producción fotovoltaica por kilowatt de capacidad [W/kWp]
track_P_mp       = ( [ f"{x}_P_mp"               for x in cases ]
    + [ "bifacial_vertical_west_P_mp",
        "bifacial_vertical_east_P_mp",
        "bifacial_south_P_mp" ] )
# Producción para cada caso
prod_n           = track_P_mp[0:5] + track_P_mp[10:]

# Ponderamos la producción fotovoltaica

# Para generación distribuida ponderamos con el área construida
prod_n_dist = [f"{x}_distributed" for x in prod_n]
a = ds[["built_surface", "REGION"]].groupby("REGION").sum().to_dataframe()
ds["sum_built_surface"] = ds["REGION"].to_dataframe().join(
    a, on = "REGION")["built_surface"].to_xarray()
ds["perc_built_surface"] = ds["built_surface"] / ds["sum_built_surface"]
ds[prod_n_dist] = ds[prod_n] * ds["perc_built_surface"]
# Podemos forzar a que la producción distribuida siempre sea hacia el sur
for v in prod_n_dist[1:]: ds[v] = ds[prod_n_dist[0]]
# Para parques solares ponderamos con las líneas de transmisión
prod_n_centr = [f"{x}_central" for x in prod_n]
ds[prod_n_centr] = ds[prod_n].where(ds["potential_solar_park_zones"])
# Unimos toda la producción
prod_n_total = [f"{x}_total" for x in prod_n]
for i, v in enumerate(prod_n_total):
    ds[v] = ( ds[prod_n_dist[i]] + ds[prod_n_centr[i]] ) / 2

# Promediamos la generación por región
ds_c = ds.groupby("REGION").mean()
# Datos agregados para generación
# Agrupamiento longitudinal
ds_i = ds.where(ds["REGION"].isin([1, 2, 4, 5, 6, 14, 19]))
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 22, 22)
ds_c_2 = ds_i.groupby("REGION").mean()
# Agrupamiento latitudinal
ds_i = ds.where(ds["REGION"].isin([1, 2, 10, 11, 12, 13]))
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 23, 23)
ds_c_3 = ds_i.groupby("REGION").mean()
# Agrupamiento total
ds_i["REGION"] = ds["REGION"].where(ds_i["REGION"] == 24, 24)
ds_c_4 = ds_i.groupby("REGION").mean()
ds_c = xr.concat([ds_c, ds_c_2, ds_c_3, ds_c_4], "REGION")
ds_c["REGION"] = ds_c["REGION"].astype(int)

delayed_obj = ds.to_netcdf(f"{dir_r}full_disc_region_2022.nc", compute = False)
with ProgressBar(): results = delayed_obj.compute()

<xarray.Dataset> Size: 40MB
Dimensions:                                  (REGION: 24, time: 8760)
Coordinates:
  * time                                     (time) datetime64[ns] 70kB 2022-...
  * REGION                                   (REGION) int64 192B 1 2 3 ... 23 24
Data variables: (12/39)
    south_no_track_P_mp                      (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    west_no_track_P_mp                       (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    east_no_track_P_mp                       (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    1_track_P_mp                             (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    2_track_P_mp                             (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    bifacial_vertical_west_P_mp              (REGION, time) float32 841kB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    ...                                       ...
    east_no_track_P_mp_total                 (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    1_track_P_mp_total                       (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    2_track_P_mp_total                       (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    bifacial_vertical_west_P_mp_total        (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    bifacial_vertical_east_P_mp_total        (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>
    bifacial_south_P_mp_total                (REGION, time) float64 2MB dask.array<chunksize=(21, 3000), meta=np.ndarray>

In [5]:
import xarray as xr
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds_2 = xr.open_dataset(f"{dir_r}full_disc_PV_2022.nc",
   ).sortby("points").chunk({"points": 2000})
ds_2["2_track_P_mp"].isel(points = 1).to_dataframe().head(24)

,points,2_track_P_mp
time,,
2022-01-01 00:00:00,13656,0.008934
2022-01-01 01:00:00,13656,0.000000
2022-01-01 02:00:00,13656,0.000000
2022-01-01 03:00:00,13656,0.000000
2022-01-01 04:00:00,13656,0.000000
2022-01-01 05:00:00,13656,0.000000
2022-01-01 06:00:00,13656,0.000000
2022-01-01 07:00:00,13656,0.000000
2022-01-01 08:00:00,13656,0.000000


In [6]:
import xarray as xr
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds_2 = xr.open_dataset(f"{dir_r}full_disc_Perez_2022.nc",
   ).sortby("points").chunk({"points": 2000})
ds_2["dni"].isel(points = 1).to_dataframe().head(24)

,points,dni
time,,
2022-01-01 00:00:00,13656,0.001000
2022-01-01 01:00:00,13656,0.001000
2022-01-01 02:00:00,13656,0.001000
2022-01-01 03:00:00,13656,0.001000
2022-01-01 04:00:00,13656,0.001000
2022-01-01 05:00:00,13656,0.001000
2022-01-01 06:00:00,13656,0.001000
2022-01-01 07:00:00,13656,0.001000
2022-01-01 08:00:00,13656,0.001000


In [ ]:
for c in np.unique(ds_i["REGION"]):
    ds_i = xr.where( ds_i["REGION"] == c,
        ds_i.shift({"time": DELTA_REGION_r(c)}), ds_i )

In [8]:
np.unique(ds["timezone"].values)

array([-8., -7., -6.])

In [ ]:
# Datos agregados para generación
# Agrupamiento longitudinal
ds_i = ds.where(ds["REGION"].isin([1, 2, 4, 5, 6, 14, 19]))
for c in np.unique(ds_i["REGION"]):
    ds_i = xr.where( ds_i["REGION"] == c,
        ds_i.shift({"time": DELTA_REGION_r(c)}), ds_i )
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 22, 22)
ds_i["hour"] = ( ds_i["time"].dt.hour
    + ds_d_i["time"].dt.minute / 60 ).copy()
ds_i["dayofyear"] = ds_i["time"].dt.dayofyear.copy()
ds_c_2 = ds_i.groupby("REGION").mean().copy()

In [ ]:
import xarray as xr

dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_dataset(f"{dir_r}full_disc_PV_2022.nc", chunks = {"time": 480})

# Número a huso horario
def TIME_REGION_r(x):
    if   x == 1 : y = -8
    elif x == 2 : y = -7
    elif x == 3 : y = -6
    elif x == 4 : y = -6
    elif x == 5 : y = -7
    elif x == 6 : y = -6
    elif x == 7 : y = -6
    elif x == 8 : y = -6
    elif x == 9 : y = -6
    elif x == 10: y = -8
    elif x == 11: y = -8
    elif x == 12: y = -8
    elif x == 13: y = -8
    elif x == 14: y = -6
    elif x == 15: y = -6
    elif x == 16: y = -6
    elif x == 17: y = -6
    elif x == 18: y = -6
    elif x == 19: y = -6
    elif x == 20: y = -6
    elif x == 21: y = -6
    return y

# Número a diferencia de huso horario
def DELTA_REGION_r(x): return -( TIME_REGION_r(x) + 8 )

# Datos agregados para generación
# Agrupamiento longitudinal
ds_i = ds.copy()
ds_i = ds_i.where(ds_i["REGION"].isin([1, 2, 4, 5, 6, 14, 19]))
for c in np.unique(ds_i["REGION"]):
    ds_i = xr.where( ds_i["REGION"] == c,
        ds_i.shift({"time": DELTA_REGION_r(c)}), ds_i )
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 22, 22)
ds_i["hour"] = ( ds_i["time"].dt.hour
    + ds_d_i["time"].dt.minute / 60 ).copy()
ds_i["dayofyear"] = ds_i["time"].dt.dayofyear.copy()
ds_c_2 = ds_i.groupby("REGION").mean().copy()
# Agrupamiento latitudinal
ds_i = ds.copy()
ds_i = ds_i.where(ds_i["REGION"].isin([1, 2, 10, 11, 12, 13]))
for c in np.unique(ds_i["REGION"]):
    ds_i = xr.where( ds_i["REGION"] == c,
        ds_i.shift({"time": DELTA_REGION_r(c)}), ds_i )
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 23, 23)
ds_i["hour"] = ( ds_i["time"].dt.hour
    + ds_d_i["time"].dt.minute / 60 ).copy()
ds_i["dayofyear"] = ds_i["time"].dt.dayofyear.copy()
ds_c_3 = ds_i.groupby("REGION").mean().copy()
# Agrupamiento total
ds_i = ds.copy()
for c in np.unique(ds_i["REGION"]):
    ds_i = xr.where( ds_i["REGION"] == c,
        ds_i.shift({"time": DELTA_REGION_r(c)}), ds_i )
ds_i["REGION"] = ds_i["REGION"].where(ds_i["REGION"] == 24, 24)
ds_i["hour"] = ( ds_i["time"].dt.hour
    + ds_d_i["time"].dt.minute / 60 ).copy()
ds_i["dayofyear"] = ds_i["time"].dt.dayofyear.copy()
ds_c_4 = ds_i.groupby("REGION").mean().copy()
ds_c = xr.concat([ds_c, ds_c_2, ds_c_3, ds_c_4], "REGION")

In [6]:
xr.open_dataset(f"{dir_r}full_disc_Perez_2022.nc",
    drop_variables = ["dhi", "Zenith_Angle", "dni"]
    ).to_netcdf(f"{dir_r}full_disc_Perez_2022_2.nc")

In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" 
    ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]


dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
if not os.path.exists(f"{dir_r}points"): os.mkdir(f"{dir_r}points")
ds_1 = xr.open_dataset(f"{dir_r}full_disc_irradiance_2022.nc",
    ).sortby("points")[["ghi", "dhi"]].chunk({"points": 1000})
ds_2 = xr.open_dataset(f"{dir_r}full_disc_physical_solar_2022.nc",
   ).sortby("points")[["Zenith_Angle", "Air_Mass"]].chunk({"points": 1000})
ds_0 = xr.merge([ds_1, ds_2])

j = 0
ds = ds_0.isel(points = slice(0*j, 0*j+1000)).load()

# Diffuse Horizontal Radiation.
ds["dni"] = ((ds["ghi"] - ds["dhi"]) /
    cos(ds["Zenith_Angle"])).astype(np.float32)
ds["dhi"] = ds["dhi"].astype(np.float32).clip(min = 0.001)
ds = ds.drop_vars("ghi")
K = 5.535e-6
# Perez clearness bins.
ds["bins"] = 0
ds["bins"] = ds["bins"].where( ds["dhi"] == 0,
    ( (ds["dhi"]+ds["dni"])/ds["dhi"] + K*ds["Zenith_Angle"]**3 )
    / ( 1 + K*ds["Zenith_Angle"]**3 ) ).astype(np.float32)
ds["dhi"] = ds["dhi"].where(ds["dhi"]>0.001, 0).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>6.200),
    8, ds["bins"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>4.500)
    & (ds["bins"]<6.200), 7, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>2.600)
    & (ds["bins"]<4.500), 6, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where((ds["bins"]>1.950)
    & (ds["bins"]<2.600), 5, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.500)
    & (ds["bins"]<1.950), 4, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.230)
    & (ds["bins"]<1.500), 3, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.065)
    & (ds["bins"]<1.500), 2, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]<1.065),
    1, ds["epsilon"] ).astype(np.float32)
Perez = pd.read_csv("../files/Perez.csv", index_col = "bin")
ds = ds.drop_vars("bins")
# Extraterrestrial radiation.
Ea = 1367
# Coeficientes
ds["Delta"] = (ds["dhi"] * ds["Air_Mass"] / Ea).astype(np.float32)
ds = ds.drop_vars("Air_Mass")
for j in Perez.columns:
    ds[j] = 0.0
    for i in Perez.index:
        ds[j] = ds[j].where(ds["epsilon"] != i,
        Perez.loc[i, j]).astype(np.float32)
    ds[j]
ds = ds.drop_vars("epsilon")
ds["F1"] = ( ds["f11"] + ds["f12"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f13"]
    ).clip(max = 0).astype(np.float32)
ds = ds.drop_vars(["f11", "f12", "f13"])
ds["F2"] = ( ds["f21"] + ds["f22"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f23"] ).astype(np.float32)
ds = ds.drop_vars(["f21", "f22", "f23", "Delta", "Zenith_Angle", "dni"])
ds.to_netcdf(f"{dir_r}points/0.nc")

Seguimiento solar



In [2]:
import xarray as xr
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" 
    ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]

print("Cargando archivos")
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"

Seguimiento solar

Cargando archivos


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from dask.diagnostics import ProgressBar

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" 
    ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]

print("Cargando archivos")
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_mfdataset([f"{dir_r}full_disc_irradiance_2022.nc",
    f"{dir_r}full_disc_physical_solar_2022.nc"], chunks = {"time": 240}
    )[["Zenith_Angle", "Air_Mass", "ghi", "dhi"]]

# Modelo de Pérez de Cielo Difuso para calcular 
# la radiación en un plano inclinado
# Diffuse Horizontal Radiation.
print("Calculando DNI")
ds["dni"] = ((ds["ghi"] - ds["dhi"]) /
    cos(ds["Zenith_Angle"])).astype(np.float32)
ds["dhi"] = ds["dhi"].astype(np.float32).clip(min = 0.001)
ds = ds.drop_vars("ghi")
K = 5.535e-6
# Perez clearness bins.
print("Calculando epsilon")
ds["bins"] = 0
ds["bins"] = ds["bins"].where( ds["dhi"] == 0,
    ( (ds["dhi"]+ds["dni"])/ds["dhi"] + K*ds["Zenith_Angle"]**3 )
    / ( 1 + K*ds["Zenith_Angle"]**3 ) ).astype(np.float32)
ds["dhi"] = ds["dhi"].where(ds["dhi"]>0.001, 0).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>6.200),
    8, ds["bins"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>4.500)
    & (ds["bins"]<6.200), 7, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>2.600)
    & (ds["bins"]<4.500), 6, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where((ds["bins"]>1.950)
    & (ds["bins"]<2.600), 5, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.500)
    & (ds["bins"]<1.950), 4, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.230)
    & (ds["bins"]<1.500), 3, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.065)
    & (ds["bins"]<1.500), 2, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]<1.065),
    1, ds["epsilon"] ).astype(np.float32)
Perez = pd.read_csv("../files/Perez.csv", index_col = "bin")
ds = ds.drop_vars("bins")
# Extraterrestrial radiation.
Ea = 1367
# Coeficientes
print(ds.nbytes/1024**3)
print("Calculando Delta")
ds["Delta"] = (ds["dhi"] * ds["Air_Mass"] / Ea).astype(np.float32)
ds = ds.drop_vars("Air_Mass")
for j in Perez.columns:
    print(ds.nbytes/1024**3)
    print(j)
    ds[j] = 0.0
    for i in Perez.index:
        ds[j] = ds[j].where(ds["epsilon"] != i,
        Perez.loc[i, j]).astype(np.float32)
    ds[j]
ds = ds.drop_vars("epsilon")
print(ds.nbytes/1024**3)
print("Calculando F1")
ds["F1"] = ( ds["f11"] + ds["f12"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f13"]
    ).clip(max = 0).astype(np.float32)
ds = ds.drop_vars(["f11", "f12", "f13"])
print(ds.nbytes/1024**3)
print("Calculando F2")
ds["F2"] = ( ds["f21"] + ds["f22"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f23"] ).astype(np.float32)
ds = ds.drop_vars(["f21", "f22", "f23", "Delta"])
print(ds.nbytes/1024**3)

In [1]:
import xarray as xr
import numpy as np

dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_mfdataset(f"{dir_r}full_disc_pv_2022.nc")
(ds.drop_vars("wind_direction").astype(np.float32) / 100).to_netcdf(f"{dir_r}full_disc_climate_2022.nc")

In [13]:
import xarray as xr
import pandas as pd
import numpy as np

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" 
    ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]

print("Cargando archivos")
dir_r = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_mfdataset(f"{dir_r}*.nc", chunks = {"time": 24}
    ).drop_vars(track_azimuth)

# Modelo de Pérez de Cielo Difuso para calcular 
# la radiación en un plano inclinado
# Diffuse Horizontal Radiation.
print("Calculando DNI")
ds["dni"] = ((ds["ghi"] - ds["dhi"]) /
    cos(ds["Zenith_Angle"])).astype(np.float32)
ds["dhi"] = ds["dhi"].astype(np.float32).clip(min = 0.001)
ds = ds.drop_vars("ghi")
K = 5.535e-6
# Perez clearness bins.
print("Calculando epsilon")
ds["bins"] = 0
ds["bins"] = ds["bins"].where( ds["dhi"] == 0,
    ( (ds["dhi"]+ds["dni"])/ds["dhi"] + K*ds["Zenith_Angle"]**3 )
    / ( 1 + K*ds["Zenith_Angle"]**3 ) ).astype(np.float32)
ds["dhi"] = ds["dhi"].where(ds["dhi"]>0.001, 0).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>6.200),
    8, ds["bins"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>4.500)
    & (ds["bins"]<6.200), 7, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>2.600)
    & (ds["bins"]<4.500), 6, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where((ds["bins"]>1.950)
    & (ds["bins"]<2.600), 5, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.500)
    & (ds["bins"]<1.950), 4, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.230)
    & (ds["bins"]<1.500), 3, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]>1.065)
    & (ds["bins"]<1.500), 2, ds["epsilon"] ).astype(np.float32)
ds["epsilon"] = xr.where( (ds["bins"]<1.065),
    1, ds["epsilon"] ).astype(np.float32)
Perez = pd.read_csv("../files/Perez.csv", index_col = "bin")
ds = ds.drop_vars("bins")
# Extraterrestrial radiation.
Ea = 1367
# Coeficientes
print("Calculando F1 y F2")
ds["Delta"] = (ds["dhi"] * ds["Air_Mass"] / Ea).astype(np.float32)
ds = ds.drop_vars("Air_Mass")
for j in Perez.columns:
    ds[j] = 0.0
    for i in Perez.index:
        ds[j] = ds[j].where(ds["epsilon"] != i, Perez.loc[i, j] )
ds = ds.drop_vars("epsilon")
ds["F1"] = ( ds["f11"] + ds["f12"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f13"]
    ).clip(max = 0).astype(np.float32)
ds = ds.drop_vars(["f11", "f12", "f13"] )
ds["F2"] = ( ds["f21"] + ds["f22"]*ds["Delta"]
    + np.radians(ds["Zenith_Angle"])*ds["f23"] ).astype(np.float32)
ds = ds.drop_vars(["f21", "f22", "f23", "Delta"])
# Iteramos para cada caso
print("Calculando POA")
for i in range(len(cases)):
    print(f"{cases[i]}")
    # Radiación difusa.
    ds["I_d"] = ( ds["dhi"] * ( (1-ds["F1"])*((1+cos(ds[track_tilt[i]]))/2)
        + ds["F1"]*cos(ds[track_AOI[i]]).clip(max = 0)/ds["b"]
        + ds["F2"]*sin(ds[track_tilt[i]]) ) ).astype(np.float32)
    ds["I_d_90"] = ( ds["dhi"] * ((1+cos(ds[track_tilt[i]]))/2)
        ).astype(np.float32)
    ds["I_d"] = ds["I_d"].where(ds["Zenith_Angle"] < 87.5,
        ds["I_d_90"]).astype(np.float32)
    ds = ds.drop_vars("I_d_90")
    ds["I_d"] = ds["I_d"].where(ds["Zenith_Angle"] < 90, 0).astype(np.float32)
    # Radiación directa.
    ds["I_b"] = (ds["dni"] * cos(ds[track_AOI[i]])).astype(np.float32)
    ds["I_b"] = ds["I_b"].where(ds[track_AOI[i]] < 90, 0).astype(np.float32)
    ds = ds.drop_vars([track_AOI[i], track_tilt[i]])
    # Radiación total en el panel.
    ds[track_POA[i]] = (ds["I_b"] + ds["I_d"]).astype(np.float32)
    ds = ds.drop_vars(["I_b", "I_d"])

ds = ds.drop_vars(["F1", "F2", "b", "Zenith_Angle",
    "Azimuth_Angle", "dhi", "dni"])

Seguimiento solar

Cargando archivos
Calculando DNI
Calculando epsilon
Calculando F1 y F2
Calculando POA
south_no_track
west_no_track
east_no_track
1_track
2_track
bifacial_vertical_west_main
bifacial_vertical_east_main
bifacial_vertical_west_back
bifacial_vertical_east_back
bifacial_south_back


In [14]:
ds

<xarray.Dataset> Size: 30GB
Dimensions:                          (points: 86193, time: 8760)
Coordinates:
  * points                           (points) int64 690kB 13653 ... 2217576
  * time                             (time) datetime64[ns] 70kB 2022-01-01 .....
Data variables: (12/18)
    latitude                         (points) float64 690kB dask.array<chunksize=(86193,), meta=np.ndarray>
    longitude                        (points) float64 690kB dask.array<chunksize=(86193,), meta=np.ndarray>
    timezone                         (points) float64 690kB dask.array<chunksize=(86193,), meta=np.ndarray>
    REGION                           (points) float32 345kB dask.array<chunksize=(86193,), meta=np.ndarray>
    potential_solar_park_zones       (points) float64 690kB dask.array<chunksize=(86193,), meta=np.ndarray>
    built_surface                    (points) float64 690kB dask.array<chunksize=(86193,), meta=np.ndarray>
    ...                               ...
    2_track_POA                      (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>
    bifacial_vertical_west_main_POA  (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>
    bifacial_vertical_east_main_POA  (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>
    bifacial_vertical_west_back_POA  (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>
    bifacial_vertical_east_back_POA  (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>
    bifacial_south_back_POA          (points, time) float32 3GB dask.array<chunksize=(86193, 24), meta=np.ndarray>

In [14]:
import xarray as xr
import numpy as np
import pandas as pd
import os

print("Slicing dataset")
print()

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_p}nsrdb_conus_irradiance_2022_crop.nc"
ds = xr.open_dataset(file_d,
    drop_variables = ["dni", "fill_flag", "time_index"], chunks = "auto"
    ).rename_dims({"phony_dim_1": "points", "phony_dim_0": "time"})

Slicing dataset



In [18]:
ds.isel(time = slice(0, None, 12), points = 
13653).load().to_dataframe().head(24)

,dhi,ghi
time,,
0,12.0,12.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0


In [12]:
import xarray as xr
import pandas as pd

dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"

ds = xr.open_dataset(f"{dir_p}full_disc_irradiance_2022.nc")
ds

<xarray.Dataset> Size: 3GB
Dimensions:  (points: 86193, time: 8760)
Coordinates:
  * points   (points) int64 690kB 13653 13656 13659 ... 2217546 2217573 2217576
  * time     (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31T23:00:00
Data variables:
    dhi      (points, time) int16 2GB ...
    ghi      (points, time) int16 2GB ...

In [ ]:
import xarray as xr
import pandas as pd

dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"

ds = xr.open_mfdataset(f"{dir_p}/points/*.nc")
ds["time"] = pd.date_range("2022-01-01", "2023-01-01", freq = "1h")[:-1]
ds.to_netcdf(f"{dir_p}full_disc_irradiance_2022.nc")

In [7]:
import xarray as xr
j = 86
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_dataset(f"{dir_p}points/points_{j}.nc")
ds

<xarray.Dataset> Size: 7MB
Dimensions:  (points: 193, time: 8760)
Coordinates:
  * points   (points) int64 2kB 2177898 2177901 2177904 ... 2217573 2217576
  * time     (time) int64 70kB 0 1 2 3 4 5 6 ... 8754 8755 8756 8757 8758 8759
Data variables:
    dhi      (points, time) int16 3MB ...
    ghi      (points, time) int16 3MB ...

In [8]:
ds.isel(points = 192).to_dataframe().head(24)

,dhi,ghi,points
time,,,
0,0,0,2217576
1,0,0,2217576
2,0,0,2217576
3,0,0,2217576
4,0,0,2217576
5,0,0,2217576
6,0,0,2217576
7,0,0,2217576
8,0,0,2217576


In [9]:
import xarray as xr
import numpy as np
import pandas as pd

print("Slicing dataset")

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_p}nsrdb_conus_irradiance_2022_crop.nc"
points = pd.read_csv(file_p, index_col = 0).sort_index().index
ds = xr.open_dataset(file_d,
    drop_variables = ["dni", "fill_flag", "time_index"], chunks = "auto"
    ).rename_dims({"phony_dim_1": "points", "phony_dim_0": "time_index"})

ds.isel(time_index = slice(0, None, 12), points = points[86192]
        ).to_dataframe().astype(np.int16).head(24)

Slicing dataset


,dhi,ghi
time_index,,
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

print("Slicing dataset")

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_p}nsrdb_conus_irradiance_2022_crop.nc"
points = pd.read_csv(file_p, index_col = 0).sort_index().index
ds = xr.open_dataset(file_d,
    drop_variables = ["dni", "fill_flag", "time_index"], chunks = "auto"
    ).rename_dims({"phony_dim_1": "points", "phony_dim_0": "time_index"})

for j in range(0, np.ceil(points.shape[0]/1000).astype(int) + 1):
    print()
    print(j*1000)

    df = ds.isel(time_index = slice(0, None, 12), points = points[0 + 1000*j]
        ).to_dataframe().astype(np.int16)

    for i in range(1 + 1000*j,1000 + 1000*j):
        if i%100==0: print(i, end = ", ")
        p = points[i]
        df = pd.concat([df, ds.isel(time_index = slice(0, None, 12),
            points = p).to_dataframe().astype(np.int16)])
        
    df["points"] = np.repeat(points[0+j*1000:1000+j*1000],
        ds["time_index"].shape[0]/12)
    df.set_index("points", append = True).to_xarray().to_netcdf(
        f"{dir_p}points/points_{j}.nc")
    
    del(df)

1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [77]:
np.ceil(points.shape[0]/1000).astype(int)

87

In [23]:
# Genera la tabla de puntos para el estudio de la curva de pato

import h5pyd
import pandas    as pd
import numpy     as np
import geopandas as gpd
import xarray    as xr
import xesmf     as xe

print("Generando tabla de puntos")
print()

# Descargamos malla con HSDS
print("Descargando malla de CONUS")
with h5pyd.File("/nrel/nsrdb/conus/nsrdb_conus_2022.h5") as f:
    print(f['time_index'])

Generando tabla de puntos

Descargando malla de CONUS
<HDF5 dataset "time_index": shape (105120,), type "|S25">


In [39]:
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
points = pd.read_csv(file_p, index_col = 0).sort_index().index

In [45]:
points[0:100]

Index([13653, 13656, 13659, 13662, 13664, 14053, 14056, 14059, 14062, 14065,
       14068, 14071, 14074, 14515, 14518, 14521, 14524, 14527, 14530, 14533,
       14536, 14539, 15088, 15091, 15094, 15097, 15102, 15105, 15108, 15111,
       15114, 15117, 15120, 15123, 15126, 15129, 15132, 15793, 15796, 15799,
       15802, 15805, 15808, 15813, 15816, 15819, 15822, 15825, 15828, 15832,
       15835, 15838, 15841, 15844, 15847, 15850, 15853, 15856, 15859, 15862,
       16695, 16698, 16701, 16704, 16707, 16710, 16713, 16716, 16719, 16722,
       16725, 16728, 16731, 16734, 16737, 16740, 16743, 16746, 16749, 16752,
       16755, 16758, 16761, 16764, 16767, 16770, 16773, 16776, 16779, 16782,
       16785, 16788, 16791, 17758, 17761, 17764, 17767, 17770, 17773, 17776],
      dtype='int64')

In [11]:
import h5py
import pandas as pd

dir_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/"
dir_p = "/datos/rodr/temp/recurso_solar/duck_curve/"
file_p = f"{dir_p}conus_points.csv"
file_d = f"{dir_n}nsrdb_conus_irradiance_2022.h5"
file_r = f"{dir_n}nsrdb_conus_irradiance_2022_crop.h5"

print("Starting partial copy of dataset")
print()

print("Loading points list")
print()
points = pd.read_csv(file_p, index_col = 0).sort_index()

Starting partial copy of dataset

Loading points list



In [19]:
import xarray as xr
import numpy as np

file_n = "/home/rodr/buffalo/rodr/Datos/NSRDB/nsrdb_conus_irradiance_2022.h5"
ds = xr.open_dataset(file_n, chunks = "auto")

In [21]:
ds

<xarray.Dataset> Size: 10TB
Dimensions:     (phony_dim_0: 105120, phony_dim_1: 2842719)
Dimensions without coordinates: phony_dim_0, phony_dim_1
Data variables:
    dhi         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    dni         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    fill_flag   (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(4000, 2000), meta=np.ndarray>
    ghi         (phony_dim_0, phony_dim_1) float64 2TB dask.array<chunksize=(8000, 2000), meta=np.ndarray>
    time_index  (phony_dim_0) <U25 11MB dask.array<chunksize=(105120,), meta=np.ndarray>
Attributes:
    version:  4.0.0

In [ ]:
ds.drop_vars(["dni", "fill_flag"]).isel({"phony_dim_0": slice(0, None, 12)}
    ).astype(np.int16).nbytes/1024**3

92.76800908148289

In [ ]:
ds.astype.nbytes/1024**3

8905.737095475197

In [13]:
# Calcula los elementos físicos de la radiación solar que dependen de la
# ubicación y del tiempo.

import pandas as pd
import numpy as np
import xarray as xr

# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

print("Seguimiento solar")
print()

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]

print("Cargando archivos")
dir_d = "/datos/rodr/temp/recurso_solar/duck_curve/"
ds = xr.open_dataset(f"{dir_d}full_disc_physical_solar_2022.nc",
    chunks = {"time": 24})[["Zenith_Angle", "Azimuth_Angle"]
    + track_tilt + track_azimuth]

# Ángulo de incidencia y valores de método de Pérez
print("Calculando AOI y Pérez a")
for i in range(len(cases)):
    print(f"Calculando {cases[i]}")
    # Ángulo entre el panel y el sol, Angle of Incidence
    ds[track_AOI[i]] = (cos(ds["Zenith_Angle"])*cos(ds[track_tilt[i]])
        + sin(ds["Zenith_Angle"])*sin(ds[track_tilt[i]])
        *cos(ds["Azimuth_Angle"]-ds[track_azimuth[i]]
        ).clip(-1, 1)).astype(np.float32)
    ds[track_AOI[i]] = acos(ds[track_AOI[i]])
    ds = ds.drop_vars([track_tilt[i], track_azimuth[i]])
    print(ds.nbytes/1024**3)

ds = ds.drop_vars(["Zenith_Angle", "Azimuth_Angle"])
print(ds.nbytes/1024**3)

Seguimiento solar

Cargando archivos
Calculando AOI y Pérez a
Calculando south_no_track
16.48949810117483
Calculando west_no_track
18.84428631514311
Calculando east_no_track
21.199074529111385
Calculando 1_track
18.843748562037945
Calculando 2_track
16.488422594964504
Calculando bifacial_vertical_west_main
18.843748547136784
Calculando bifacial_vertical_east_main
21.199074499309063
Calculando bifacial_vertical_west_back
23.554400451481342
Calculando bifacial_vertical_east_back
25.90972640365362
Calculando bifacial_south_back
28.2645146176219
23.553862683475018


In [14]:
ds

<xarray.Dataset> Size: 25GB
Dimensions:                                         (points: 72175, time: 8760)
Coordinates:
  * points                                          (points) int64 577kB 1881...
  * time                                            (time) datetime64[ns] 70kB ...
Data variables:
    south_no_track_Angle_of_Incidence               (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    west_no_track_Angle_of_Incidence                (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    east_no_track_Angle_of_Incidence                (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    1_track_Angle_of_Incidence                      (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    2_track_Angle_of_Incidence                      (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_west_main_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_east_main_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_west_back_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_vertical_east_back_Angle_of_Incidence  (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>
    bifacial_south_back_Angle_of_Incidence          (points, time) float32 3GB dask.array<chunksize=(72175, 24), meta=np.ndarray>

In [10]:
ds.nbytes/1024**3

49.467471508309245

In [1]:
 # Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi


In [7]:
TZ = ds["timezone"]
lat = ds["latitude"]
lon = ds["longitude"]
a = ((sin(ds["Declination"])
    - cos(ds["Zenith_Angle"])*sin(lat))
    / (sin(ds["Zenith_Angle"])*cos(lat)))

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

In [22]:
b =sin(ds["Zenith_Angle"])*cos(lat)

In [27]:
(ds["Hour_Angle"]==0).sum()

<xarray.DataArray 'Hour_Angle' ()> Size: 8B
array(0)

In [24]:
b.min()

<xarray.DataArray ()> Size: 8B
array(-0.00016722)

In [1]:
import h5pyd
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# Funciones trigonométricas.
def sin(x) : return np.sin(np.radians(x))
def cos(x) : return np.cos(np.radians(x))
def tan(x) : return np.tan(np.radians(x))
def asin(x): return np.arcsin(x) * 180/np.pi
def acos(x): return np.arccos(x) * 180/np.pi
def atan(x): return np.arctan(x) * 180/np.pi

# Funciones para claves de regiones eléctricas

# Clave a número
def NUM_REGION(x):
    if   x == "BCA"  : y = 1
    elif x == "BCS"  : y = 2
    elif x == "CEN"  : y = 3
    elif x == "NES"  : y = 4
    elif x == "NOR"  : y = 5
    elif x == "NTE"  : y = 6
    elif x == "OCC"  : y = 7
    elif x == "ORI"  : y = 8
    elif x == "PEN"  : y = 9
    elif x == "PGE"  : y = 10
    elif x == "SCE"  : y = 11
    elif x == "SDGE" : y = 12
    elif x == "VEA"  : y = 13
    elif x == "COAST": y = 14
    elif x == "EAST" : y = 15
    elif x == "FWEST": y = 16
    elif x == "NORTH": y = 17
    elif x == "NCENT": y = 18
    elif x == "SOUTH": y = 19
    elif x == "SCENT": y = 20
    elif x == "WEST" : y = 21
    elif x == "LON"  : y = 22
    elif x == "LAT"  : y = 23
    elif x == "TOTAL": y = 24
    return y

# Número a clave
def NUM_REGION_r(x):
    if   x == 1 : y = "BCA"
    elif x == 2 : y = "BCS"
    elif x == 3 : y = "CEN"
    elif x == 4 : y = "NES"
    elif x == 5 : y = "NOR"
    elif x == 6 : y = "NTE"
    elif x == 7 : y = "OCC"
    elif x == 8 : y = "ORI"
    elif x == 9 : y = "PEN"
    elif x == 10: y = "PGE"
    elif x == 11: y = "SCE"
    elif x == 12: y = "SDGE"
    elif x == 13: y = "VEA"
    elif x == 14: y = "COAST"
    elif x == 15: y = "EAST"
    elif x == 16: y = "FWEST"
    elif x == 17: y = "NORTH"
    elif x == 18: y = "NCENT"
    elif x == 19: y = "SOUTH"
    elif x == 20: y = "SCENT"
    elif x == 21: y = "WEST"
    elif x == 22: y = "LON"
    elif x == 23: y = "LAT"
    elif x == 24: y = "TOTAL"
    return y

# Número a nombre
def NAME_REGION_r(x):
    if   x == 1 : y = "Baja California"
    elif x == 2 : y = "Baja California Sur"
    elif x == 3 : y = "Centro"
    elif x == 4 : y = "Noreste"
    elif x == 5 : y = "Noroeste"
    elif x == 6 : y = "Norte"
    elif x == 7 : y = "Occidente"
    elif x == 8 : y = "Oriente"
    elif x == 9 : y = "Peninsular"
    elif x == 10: y = "Pacific G&E"
    elif x == 11: y = "Southern Cal Edison"
    elif x == 12: y = "San Diego G&E"
    elif x == 13: y = "Valley Electric Assn"
    elif x == 14: y = "Coast"
    elif x == 15: y = "East"
    elif x == 16: y = "Far West"
    elif x == 17: y = "North"
    elif x == 18: y = "North Central"
    elif x == 19: y = "South"
    elif x == 20: y = "South Central"
    elif x == 21: y = "West"
    elif x == 22: y = "Longitudinal"
    elif x == 23: y = "Latitudinal"
    elif x == 24: y = "Total"
    return y

# Número a huso horario
def TIME_REGION_r(x):
    if   x == 1 : y = -8
    elif x == 2 : y = -7
    elif x == 3 : y = -6
    elif x == 4 : y = -6
    elif x == 5 : y = -7
    elif x == 6 : y = -6
    elif x == 7 : y = -6
    elif x == 8 : y = -6
    elif x == 9 : y = -6
    elif x == 10: y = -8
    elif x == 11: y = -8
    elif x == 12: y = -8
    elif x == 13: y = -8
    elif x == 14: y = -6
    elif x == 15: y = -6
    elif x == 16: y = -6
    elif x == 17: y = -6
    elif x == 18: y = -6
    elif x == 19: y = -6
    elif x == 20: y = -6
    elif x == 21: y = -6
    return y

# Número a diferencia de huso horario
def DELTA_REGION_r(x): return -( TIME_REGION_r(x) + 8 )

In [8]:
dir_d = "/home/rodr/temp/recurso_solar/duck_curve/"
points = pd.read_csv(f"{dir_d}/conus_points.csv", index_col = 0)
points = points[points["REGION"].notnull()]
points["REGION"] = points["REGION"].astype(np.int16)
ds = points[["latitude", "longitude", "elevation", "timezone", "REGION",
    "potential_solar_park_zones", "built_surface"]
    ].to_xarray().rename({"index": "points"})
ds["time"] = pd.date_range("2022-01-01", "2023-01-01", freq = "1h")[:-1]
ds = ds.chunk(time = 24)
ds["Pressure"] = (1013.25 * (1 - 0.0065/288.15*ds["elevation"]) ** 5.25588
    ).astype(np.float32)
ds = ds.drop_vars("elevation")
ds

<xarray.Dataset> Size: 4MB
Dimensions:                     (points: 72175, time: 8760)
Coordinates:
  * points                      (points) int64 577kB 1881061 1884568 ... 21809
  * time                        (time) datetime64[ns] 70kB 2022-01-01 ... 202...
Data variables:
    latitude                    (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    longitude                   (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    timezone                    (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    REGION                      (points) int16 144kB dask.array<chunksize=(72175,), meta=np.ndarray>
    potential_solar_park_zones  (points) int64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    built_surface               (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    Pressure                    (points) float32 289kB dask.array<chunksize=(72175,), meta=np.ndarray>

In [9]:
# Convertimos a fecha.
ds["hour"] = (ds["time"].dt.hour + ds["time"].dt.minute/60).astype(np.float32)
ds["dayofyear"] = ds["time"].dt.dayofyear.astype(np.int16)

TZ = ds["timezone"]
lat = ds["latitude"]
lon = ds["longitude"]

# Eccentric anomaly of the earth in its orbit around the sun.
ds["Day_Angle"] = (6.283185 * ( ds["dayofyear"] - 1 ) / 365).astype(np.float32)
# Declinación.
ds["Declination"] = ( ( 0.006918 - 0.399912 * np.cos(ds["Day_Angle"])
    + 0.070257*np.sin(ds["Day_Angle"])
    - 0.006758*np.cos(2*ds["Day_Angle"])
    + 0.000907*np.sin(2*ds["Day_Angle"])
    - 0.002697*np.cos(3*ds["Day_Angle"])
    + 0.00148*np.sin(3*ds["Day_Angle"]) ) * 180/np.pi ).astype(np.float32)
# Ecuación del tiempo.
ds["Time_Equation"] = ( ( 0.000075 + 0.001868*np.cos(ds["Day_Angle"])
    - 0.032077*np.sin(ds["Day_Angle"])
    - 0.014615*np.cos(2*ds["Day_Angle"])
    -0.040849*np.sin(2*ds["Day_Angle"])) * 229.18 ).astype(np.float32)
# Longitud del punto subsolar.
ds["lon_subs"] = -15 * ( ds["hour"] - TZ
    + ds["Time_Equation"]/60 ).astype(np.float32)
# Ángulo horario.
ds["Hour_Angle"] = ( 15 * ( ds["hour"] - 12
    - ds["Time_Equation"]/60 + ((lon-TZ*15)*4)/60 ) ).astype(np.float32)
ds = ds.drop_vars(["Time_Equation"])
# Posiciones del analema solar.
ds["Sz"] = ( sin(lat)*sin(ds["Declination"])
    - cos(lat)*cos(ds["Declination"])
    *cos(ds["lon_subs"]-lon) ).astype(np.float32)
ds = ds.drop_vars(["lon_subs"])
# Ángulo del cénit solar.
ds["Zenith_Angle"] = acos(ds["Sz"]).astype(np.float32)
ds = ds.drop_vars("Sz")
# Ángulo acimutal solar.
ds["Azimuth_Angle"] = acos( ( sin(ds["Declination"])
    - cos(ds["Zenith_Angle"])*sin(lat) )
    / ( sin(ds["Zenith_Angle"])*cos(lat) ) ).astype(np.float32)
ds["Azimuth_Angle"] = ds["Azimuth_Angle"].where(
    ds["Hour_Angle"] < 0, 360 - ds["Azimuth_Angle"] )
ds = ds.drop_vars(["Declination", "Hour_Angle"])
# Masa de aire.
ds["Air_Mass"] = ( 1/(cos(ds["Zenith_Angle"])
    + 0.15/(93.885 - ds["Zenith_Angle"])**1.253 )
    * ds["Pressure"]/1013.25 ).astype(np.float32)
ds["Air_Mass"] = ds["Air_Mass"].where( ds["Zenith_Angle"] < 85.5, 0 )
ds = ds.drop_vars(["Day_Angle", "Pressure"])

In [6]:
# Casos de orientación de sistemas fotovoltaico

# Casos a estudiar
cases = [ "south_no_track", "west_no_track", "east_no_track",
    "1_track", "2_track", "bifacial_vertical_west_main",
    "bifacial_vertical_east_main", "bifacial_vertical_west_back",
    "bifacial_vertical_east_back", "bifacial_south_back" ]
# Variables fotovoltaicas por caso
# Inclinación
track_tilt       =   [ f"{x}_Tilt"               for x in cases ]
# Azimuth
track_azimuth    =   [ f"{x}_Azimuth"            for x in cases ]
# Ángulo entre el panel y el sol, Angle of Incidence
track_AOI        =   [ f"{x}_Angle_of_Incidence" for x in cases ]
# Radiación incidente en el panel [W/m^2], Plane of Array Irradiace
track_POA        =   [ f"{x}_POA"                for x in cases ]
# Producción fotovoltaica por kilowatt de capacidad [W/kWp]
track_P_mp       = ( [ f"{x}_P_mp"               for x in cases ]
    + [ "bifacial_vertical_west_P_mp",
        "bifacial_vertical_east_P_mp",
        "bifacial_south_P_mp" ] )
# Producción para cada caso
prod_n           = track_P_mp[0:5] + track_P_mp[10:]
# Factor bifacial
P_bf = [ 1, 1, 1, 1, 1, 1, 1, 0.7, 0.7, 0.7 ]

# Ángulos de orientación de los sistemas
# Orientación del seguidor de un eje
# Asumimos inclinación de 0 grados
azimuth_tracker = 180
# south_no_track
ds[track_azimuth[0]] = 180
ds[track_tilt[0]   ] = lat
# west_no_track
ds[track_azimuth[1]] = 270
ds[track_tilt[1]   ] = lat
# east_no_track
ds[track_azimuth[2]] = 90
ds[track_tilt[2]   ] = lat
# 1_track
ds[track_tilt[3]   ] = np.abs( atan( tan(ds["Zenith_Angle"])
    * sin(ds["Azimuth_Angle"] - azimuth_tracker) ) ).astype(np.float32)
ds[track_azimuth[3]] = 90
ds[track_azimuth[3]] = ds["1_track_Azimuth"
    ].where(ds["Azimuth_Angle"]<180, 270).astype(np.float32)
# 2_track
ds[track_tilt[4]   ] = ds["Zenith_Angle"]
ds[track_azimuth[4]] = ds["Azimuth_Angle"]
# bifacial_vertical_west_main
ds[track_tilt[5]   ] = 90
ds[track_azimuth[5]] = 270
# bifacial_vertical_east_main
ds[track_tilt[6]   ] = 90
ds[track_azimuth[6]] = 90
# bifacial_vertical_west_back
ds[track_tilt[7]   ] = 90
ds[track_azimuth[7]] = 90
# bifacial_vertical_east_back
ds[track_tilt[8]   ] = 90
ds[track_azimuth[8]] = 270
# bifacial_south_back
ds[track_tilt[9]   ] = 90 + lat
ds[track_azimuth[9]] = 0

In [7]:
print(ds.nbytes/1024**3)
ds = ds.transpose("points", "time").chunk(time = 24)
ds

16.492907969281077


<xarray.Dataset> Size: 18GB
Dimensions:                              (points: 72175, time: 8760)
Coordinates:
  * points                               (points) int64 577kB 1881061 ... 21809
  * time                                 (time) datetime64[ns] 70kB 2022-01-0...
Data variables: (12/31)
    latitude                             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    longitude                            (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    timezone                             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    REGION                               (points) int16 144kB dask.array<chunksize=(72175,), meta=np.ndarray>
    potential_solar_park_zones           (points) int64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    built_surface                        (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    ...                                   ...
    bifacial_vertical_west_back_Tilt     int64 8B 90
    bifacial_vertical_west_back_Azimuth  int64 8B 90
    bifacial_vertical_east_back_Tilt     int64 8B 90
    bifacial_vertical_east_back_Azimuth  int64 8B 270
    bifacial_south_back_Tilt             (points) float64 577kB dask.array<chunksize=(72175,), meta=np.ndarray>
    bifacial_south_back_Azimuth          int64 8B 0